In [1]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('tmdb_movies_data.csv')


In [40]:
df = df[['id','original_title','cast','director','keywords','overview','genres','release_year','vote_average']]

In [41]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [42]:
df.drop(df[df['original_title'] == 'Cazuza - O tempo nÃ£o pÃ¡ra'].index, inplace= True)
df.drop(df[df['original_title'] == 'í•˜ìš¸ë§'].index, inplace= True)
df = df.reset_index(drop=True)

In [43]:
def remove(obj):
    return obj.split('|')

In [7]:
df['cast'] = df['cast'].apply(lambda x:remove(str(x)))

In [8]:
df['keywords'] = df['keywords'].apply(lambda x:remove(str(x)))
df['genres'] = df['genres'].apply(lambda x:remove(str(x)))


In [9]:
rec_df = df

In [10]:
df[df['id'] == 355338]

,id,original_title,cast,director,keywords,overview,genres,release_year,vote_average
251,355338,Riley's First Date?,"[Amy Poehler, Phyllis Smith, Bill Hader, Lewis...",Josh Cooley,"[mother daughter relationship, rock music, gir...","Riley, now 12, is hanging out at home with her...","[Animation, Family]",2015,7.3


In [11]:
rec_df['genres'] = rec_df['genres'].apply(lambda x:x[0])

In [12]:
rec_df = rec_df[['id','original_title','genres','vote_average']]

In [13]:
import pickle
pickle.dump(rec_df,open('movie_data.sav','wb'))

In [38]:
df['cast'] = df['cast'].apply(lambda x:x[:4])
df['cast']


0       [ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...
1       [TomHardy, CharlizeTheron, HughKeays-Byrne, Ni...
2       [ShaileneWoodley, TheoJames, KateWinslet, Anse...
3       [HarrisonFord, MarkHamill, CarrieFisher, AdamD...
4       [VinDiesel, PaulWalker, JasonStatham, Michelle...
                              ...                        
9298    [MichaelHynson, RobertAugust, Lord'TallyHo'Ble...
9299    [JamesGarner, EvaMarieSaint, YvesMontand, Tosh...
9300    [InnokentiySmoktunovskiy, OlegEfremov, GeorgiZ...
9301    [TatsuyaMihashi, AkikoWakabayashi, MieHama, Jo...
9302    [HaroldP.Warren, TomNeyman, JohnReynolds, Dian...
Name: cast, Length: 9303, dtype: object

In [15]:
df['cast'] = df['cast'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [16]:
df['director'] = df['director'].apply(lambda x:[str(x).replace(' ','')])

In [17]:
df['keywords'] = df['keywords'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [18]:
df['genres'] = df['genres'].apply(lambda x:[str(i).replace(' ','') for i in x])

In [19]:
df['overview'] = df['overview'].apply(lambda x:str(x).split(' '))

In [20]:
df['release_year'] = df['release_year'].apply(lambda x:[str(x)])

In [21]:
df['corpus'] = df['overview'] + df['keywords'] + df['genres'] + df['cast'] + df['director'] + df['release_year']

In [22]:
new_df = df[['id','original_title','corpus','vote_average']]

In [23]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))

/tmp/ipykernel_27279/3919970262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: " ".join(x))


In [24]:
new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))

/tmp/ipykernel_27279/420603864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['corpus'] = new_df['corpus'].apply(lambda x: str.lower(x))


In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stem(text):
    words = []
    for i in text.split():
        words.append(ps.stem(i))
    
    return " ".join(words)

In [27]:
# new_df['corpus'] = new_df['corpus'].apply(stem)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

# Stop Words = a,the,of,after,now

In [29]:
cv = CountVectorizer(stop_words='english',min_df=2)

In [30]:
data = cv.fit_transform(new_df['corpus']).toarray()

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity = cosine_similarity(data)

In [33]:
new_df = new_df.rename({'original_title':'title'},axis=1)

In [34]:
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:11]

[(4395, 0.27931019386546413),
 (8695, 0.23911404992940524),
 (8666, 0.22156468376279895),
 (5701, 0.18181818181818185),
 (405, 0.16283473681973243),
 (37, 0.15974461276617435),
 (226, 0.14083575804390608),
 (6635, 0.1371450962647483),
 (7723, 0.13483997249264842),
 (85, 0.13273296830047474)]

In [35]:
df.iloc[0]

id                                                           135397
original_title                                       Jurassic World
cast              [ChrisPratt, BryceDallasHoward, IrrfanKhan, Vi...
director                                           [ColinTrevorrow]
keywords          [monster, dna, tyrannosaurusrex, velociraptor,...
overview          [Twenty-two, years, after, the, events, of, Ju...
genres                                           [A, c, t, i, o, n]
release_year                                                 [2015]
vote_average                                                    6.5
corpus            [Twenty-two, years, after, the, events, of, Ju...
Name: 0, dtype: object

In [36]:
new_df.reset_index(drop=True)
pickle.dump(new_df,open('data.sav','wb'))
pickle.dump(similarity,open('similarity.sav','wb'))
